In [20]:
import copy
from venv import create

class CliffWalkingEnv:
    """ 悬崖漫步环境"""
    def __init__(self, ncol = 12, nrow = 4):
        self.ncol = ncol# 定义网格世界的列
        self.nrow = nrow# 定义网格世界的行
        self.P = self.createP()# 转移矩阵P[state][action] = [(p, next_state, reward, done)]包含下一个状态和奖励
    
    def createP(self):
        P = [[[] for j in range(4)] for i in range(self.nrow * self.ncol)]#* 初始化
        change = [[0, -1], [0, 1], [-1, 0], [1, 0]]# 4种动作, change[0]:上,change[1]:下, change[2]:左, change[3]:右。坐标系原点(0,0)定义在左上角
        for i in range(self.nrow):# 循环，构建P
            for j in range(self.ncol):#
                for a in range(4):#
                    if i == self.nrow - 1 and j > 0:# 位置在悬崖或者目标状态,因为无法继续交互,任何动作奖励都为0
                        P[i * self.ncol + j][a] = [(1, i * self.ncol + j, 0, True)]#*
                        continue#
                    next_x = min(self.ncol - 1, max(0, j + change[a][0]))# 其他位置
                    next_y = min(self.nrow - 1, max(0, i + change[a][1]))#
                    next_state = next_y * self.ncol + next_x#
                    reward = -1#
                    done = False#
                    if next_y == self.nrow - 1 and next_x > 0:# 下一个位置在悬崖或者终点
                        done = True#
                        if next_x < self.ncol - 1:# 下一个位置在悬崖
                            reward = -100#
                    P[i * self.ncol + j][a] = [(1, next_state, reward, done)]#*
        return P#


In [40]:
class PolicyIteration:
    """ 策略迭代算法 """
    def __init__(self, env, theta, gamma):
        self.env = env#
        self.v = [0] * self.env.ncol * self.env.nrow# 初始化价值为0
        self.pi = [[1 / 4] * 4 for s in range(self.env.ncol * self.env.nrow)]# 初始化策略为均匀策略
        self.gamma = gamma# 折扣因子
        self.theta = theta# 价值收敛阈值

    def policy_evaluation(self):
        """ 策略评估 """
        cnt =  1# 计数器
        while 1:# 循环，开始策略评估
            max_diff = 0#
            new_v = [0] * self.env.ncol * self.env.nrow#*
            for s in range(self.env.ncol * self.env.nrow):# 循环，遍历每一个状态
                qsa_list = []# 开始计算状态s下的所有Q(s,a)价值
                for a in range(4):# 循环，遍历每一个动作
                    qsa = 0# 状态动作价值
                    for res in self.env.P[s][a]:# 循环，遍历P的每个元素
                        p, next_state, r, done = res# 取出转移概率、下一个状态、奖励， 完成状态
                        qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))# 计算状态s使用策略pi下的动作价值函数，本章环境比较特殊,奖励和下一个状态有关,所以需要和状态转移概率相乘
                    qsa_list.append(qsa)#* 扩展qsa_list表
                new_v[s] = sum([self.pi[s][a] * qsa_list[a] for a in range(4)])# 状态价值函数和动作价值函数之间的关系
                max_diff = max(max_diff, abs(new_v[s] - self.v[s]))# 计算新旧价值函数的差值
            self.v = new_v#
            if max_diff < self.theta:# 满足收敛条件,退出评估迭代
                break#
            cnt += 1# 增加计数
        print("策略评估进行%d轮后完成" % cnt)

    def policy_improvement(self):
        """ 策略提升 """
        for s in range(self.env.ncol * self.env.nrow):# 循环，遍历每个状态
            qsa_list = []# 开始计算状态s下的所有Q(s,a)价值
            for a in range(4):# 循环，计算每个动作的价值
                qsa = 0#
                for res in self.env.P[s][a]:#* 循环，遍历P的每个元素
                    p, next_state, r, done = res# 
                    qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))# 计算动作价值
                qsa_list.append(qsa)# 扩展qsa_list列表
            maxq = max(qsa_list)# 找出最大的动作价值
            cntq = qsa_list.count(maxq)# 计算有几个动作得到了最大的Q值
            self.pi[s] = [1 / cntq if qsa_list[a] == maxq else 0 for a in range(4)]# 让这些动作均分概率
        print("策略提升完成")
        return self.pi# 返回策略
    
    def policy_iteration(self):
        """ 策略迭代 """
        while 1:# 循环
            self.policy_evaluation()# 策略评估
            old_pi = copy.copy(self.pi)# 将列表进行深拷贝,方便接下来进行比较
            new_pi = self.policy_improvement()# 策略提升得到新策略
            if old_pi == new_pi:# 比较新旧策略
                break# 如果相同,则退出循环

In [41]:
def print_agent(agent, action_meaning, disaster=[], end=[]):
    print("状态价值：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            # 为了输出美观,保持输出6个字符
            print('%6.6s' % ('%.3f' % agent.v[i * agent.env.ncol + j]), end=' ')
        print()

    print("策略：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            # 一些特殊的状态,例如悬崖漫步中的悬崖
            if (i * agent.env.ncol + j) in disaster:
                print('****', end=' ')
            elif (i * agent.env.ncol + j) in end:  # 目标状态
                print('EEEE', end=' ')
            else:
                a = agent.pi[i * agent.env.ncol + j]
                pi_str = ''
                for k in range(len(action_meaning)):
                    pi_str += action_meaning[k] if a[k] > 0 else 'o'
                print(pi_str, end=' ')
        print()

env = CliffWalkingEnv()
action_meaning = ['^', 'v', '<', '>']
theta = 0.001
gamma = 0.9
agent = PolicyIteration(env, theta, gamma)
agent.policy_iteration()
print_agent(agent, action_meaning, list(range(37, 47)), [47])

策略评估进行60轮后完成
策略提升完成
策略评估进行72轮后完成
策略提升完成
策略评估进行44轮后完成
策略提升完成
策略评估进行12轮后完成
策略提升完成
策略评估进行1轮后完成
策略提升完成
状态价值：
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


In [50]:
class ValueIteration:
    """ 价值迭代算法 """
    def __init__(self, env, theta, gamma):
        self.env = env
        self.v = [0] * self.env.ncol * self.env.nrow  # 初始化价值为0
        self.theta = theta  # 价值收敛阈值
        self.gamma = gamma
        self.pi = [None for i in range(self.env.ncol * self.env.nrow)]# 价值迭代结束后得到的策略

    def value_iteration(self):
        cnt = 0# 
        while 1:#
            max_diff = 0#
            new_v = [0] * self.env.ncol * self.env.nrow#*
            for s in range(self.env.ncol * self.env.nrow):#
                qsa_list = []#*
                for a in range(4):#
                    qsa = 0#
                    for res in self.env.P[s][a]:#
                        p, next_state, r, done = res#
                        qsa += p * (r + gamma * self.v[next_state] * (1 - done))#
                    qsa_list.append(qsa)#
                new_v[s] = max(qsa_list)#* 这一行是价值迭代和策略迭代的主要区别
                max_diff = max(max_diff, abs(self.v[s] - new_v[s]))#*
            self.v = new_v#*
            if max_diff < self.theta:#
                break#
            cnt += 1#
        print("价值迭代一共进行%d轮" % cnt)
        self.get_policy()

    def get_policy(self):
        """根据价值函数导出一个贪婪策略"""
        for s in range(self.env.ncol * self.env.nrow):# 循环，遍历每个状态
            qsa_list = []# 初始化状态动作价值表
            for a in range(4):# 循环，遍历每个动作
                qsa = 0# 初始化状态动作价值
                for res in self.env.P[s][a]:# 循环，遍历每个转移概率
                    p, next_state, r, done = res# 记录相关数据
                    qsa += p * (r + gamma * self.v[next_state] * (1 - done))# 计算状态动作价值
                qsa_list.append(qsa)# 扩展状态动作价值表
            max_q = max(qsa_list)# 找到状态下价值最大的动作
            cont_q = qsa_list.count(max_q)# 计算有几个动作得到了最大的Q值
            self.pi[s] = [1/cont_q if qsa_list[a] == max_q else 0 for a in range(4)]# 让这些动作均分概率

env = CliffWalkingEnv()# 实例化环境
theta = 0.001
gamma = 0.9
agent = ValueIteration(env, theta, gamma)# 实例化智能体
agent.value_iteration()# 智能体价值迭代
action_meaning = ['^', 'v', '<', '>']
print_agent(agent, action_meaning, list(range(37, 47)), [47])

价值迭代一共进行14轮
状态价值：
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


In [ ]:
import gym
env = gym.make("FrozenLake-v1")# 创建环境"FrozenLake-v0"
env = env.unwrapped# 解封装才能访问状态转移矩阵P
env.render()# 环境渲染,通常是弹窗显示或打印出可视化的环境

holes = set()# 初始化洞的位置
ends = set()# 初始化结束位置
for s in env.P: # 循环，遍历状态
    for a in env.P[s]:# 循环，遍历动作
        for s_ in env.P[s][a]:# 循环，遍历下一个状态
            if s_[2] == 1:# 如果获得奖励s_[2]为1,代表是目标
                ends.add(s_[1])#& 将目标的索引s_[1]加入ends
            if s_[3] == True:# 如果下一个状态是终止状态s_[3]
                holes.add(s_[1])# 将下一个状态的索引s_[1]加入holes
holes = holes - ends#* 从洞的集合中删去结束的索引

print("冰洞的索引:", holes)
print("目标的索引:", ends)

for a in env.P[s]:# 循环，查看目标左边一格的状态转移信息
    print(env.P[14][a])

冰洞的索引: {11, 12, 5, 7}
目标的索引: {15}
[(0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False)]
[(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True)]
[(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)]
[(0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False)]


In [63]:
action_meaning = ['<', 'v', '>', '^'] # 这个动作意义是Gym库针对冰湖环境事先规定好的
theta = 1e-5
gamma = 0.9
agent = PolicyIteration(env, theta, gamma)
agent.policy_iteration()
print_agent(agent, action_meaning, [5, 7, 11, 12], [15])

策略评估进行25轮后完成
策略提升完成
策略评估进行58轮后完成
策略提升完成
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 


In [64]:
action_meaning = ['<', 'v', '>', '^']
theta = 1e-5
gamma = 0.9
agent = ValueIteration(env, theta, gamma)
agent.value_iteration()
print_agent(agent, action_meaning, [5, 7, 11, 12], [15])

价值迭代一共进行60轮
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 
